<a href="https://colab.research.google.com/github/anhadh3101/HAR/blob/main/HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd

import os
from PIL import Image
import matplotlib.pyplot as plt
import torch
import tensorflow as tf
from torchvision import transforms
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Dropout, Flatten
from keras import Model
from keras.models import Sequential
from keras.applications.vgg16 import VGG16

# Loading the dataset
1. Add the training dataset's zip file to your google drive.
2. Mount the google drive to the colab notebook by running the first code setion.
3. Unzip the file to the local environment by running the second code section. (Set the path to the dataset accordingly)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/archive.zip

# Preprocessing the data

Preprocess the training data to convert the images and labels into Numpy arrays for CNN to recognize.

In [ ]:
# Getting paths to training and testing images
train_fol = '/content/Human Action Recognition/train'
test_fol = '/content/Human Action Recognition/test'

# Getting paths to csv files
train_csv = '/content/Human Action Recognition/Training_set.csv'
test_csv = '/content/Human Action Recognition/Testing_set.csv'

# Store the csv file as a Pandas dataframe
train_data = pd.read_csv(train_csv)

# Get the columns of the dataframe
train_filenames = train_data['filename']
train_labels = train_data['label']

In [ ]:
# Getting the paths to test images
test_data = pd.read_csv(test_csv)
test_filenames = test_data['filename']
test_paths = []
for i in range(len(test_data)):
  test_paths.append(test_fol + '/' + test_filenames[i])

In [ ]:
def img_splitter(img_path):
  img = Image.open(img_path) # Get the image
  # Split the image into its red, green and blue components and store it in an array
  return np.asarray(img.resize((160, 160))) # Convert and store the image

In [ ]:
# Store the preprocessed arrays of images
converted_images = []

# Go through each image and store the corresponding array
for i in range(len(train_data)):
  img = Image.open(os.path.join(train_fol + "/" + train_filenames[i])) # Get the image
  # Split the image into its red, green and blue components and store it in an array
  converted_images.append(np.asarray(img.resize((160, 160)))) # Convert and store the image

# Convert the array into a Numpy array
x_train = np.asarray(converted_images)

In [ ]:
# Convert the labels column in the dataframe to an array with 15 indices for each label
y_train = to_categorical(np.asarray(train_labels.factorize()[0]))

# Using Pretrained Models

## 1. ResNet101

In [ ]:
# Intializing a Sequential model
resnet101_model = Sequential()

# Getting a pretrained ResNet101 model
pretrained_resnet101_model = tf.keras.applications.ResNet101(
  include_top=False,
  input_shape=(160, 160, 3),
  pooling='avg',
  classes=15, # Initialized the number of classes in the dataset
  weights='imagenet'
)

# Fixing the weights of the pretrained model to avoid retraining
for layer in pretrained_resnet101_model.layers:
  layer.trainable = False

# Adding the pretrained model and additional layers to the Sequential model
resnet101_model.add(pretrained_resnet101_model)
resnet101_model.add(Flatten())
resnet101_model.add(Dense(512, activation='relu'))
resnet101_model.add(Dense(15, activation='softmax')) # Predicts one of 15 labels

In [ ]:
# Compiling the model
resnet101_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet101_model.summary()

In [ ]:
# Training the model
history = resnet101_model.fit(x_train, y_train, epochs=60)

In [ ]:
test_img = Image.open(test_paths[9])
a = np.asarray(test_img.resize((160, 160)))
result = resnet101_model.predict(np.asarray([a]))
item_index = np.where(result==np.max(result))
print(item_index[1][0])

test_img

Failed Code for Alexnet

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
def get_prediction(img_path, model):
  inp_image = Image.open(img_path)
  inp_tensor = preprocess(inp_image)
  inp_batch = inp_tensor.unsqueeze(0)
  # with torch.no_grad():
  return model(inp_batch)

In [ ]:
prediction = get_prediction(test_paths[0], alexnet_model)
probabilities = torch.nn.functional.softmax(prediction[0], dim=0)
print(probabilities)